
# <p style="background-color:white;font-family:seoge print;color:slateblue ;font-size:170%;text-align:center;border-radius:20px 60px;">EYE FOR BLIND</p>

![](https://images.immediate.co.uk/production/volatile/sites/4/2018/08/iStock_50575182_LARGE-b054f52-e1533553873292.jpg?quality=90&resize=768,574)

**Please Note :** *Google Colab has been used to train the model*


# <p style="background-color:white;font-family:seoge print;color:gray ;font-size:170%;text-align:center;border-radius:20px 60px;">INTRODUCTION</p>



#### The World Health Organization (WHO) has reported that approximately 285 million people are visually impaired worldwide, and out of these 285 million, 39 million are completely blind. It gets extremely tough for them to carry out daily activities, one of which is reading. 

    From reading a newspaper or a magazine to reading an important text message from your bank, it is tough for them to read the text written in it.A similar problem they also face is seeing and enjoying the beauty of pictures and images. Today, in the world of social media, millions of images are uploaded daily. Some of them are about your friends and family, while some of them are about nature and its beauty. Understanding what is present in that image is quite a challenge for certain people who are suffering from visual impairment or who are blind.


*In an initiative to help them experience the beauty of the images, Facebook had earlier launched a unique feature earlier that can help blind people operate the app on their mobile phones. The feature could explain the contents of an image that their friends have posted on Facebook. So, say, if someone posted a picture with their dog in the park, the application would speak out the contents and may describe it like, “This image may contain a dog standing with a man around the trees.”*

**Here, in this kernel will learn how to make a model that is similar to the one developed by Facebook, specifically such that a blind person knows the contents of an image in front of them with the help of a CNN-RNN based model. The model will convert the contents of an image and will provide the output in the form of audio.**



    In this project, we will convert an image to text description first; then, using a simple text-to-speech API, we will extract the text description/caption and convert it to audio. So the central part is focused on building the caption/text description whereas the second part, which is transforming the text to speech is relatively easy with the text to speech API. Once the model is built, we will deploy the project on our local system using a Flask-based model to generate audio-based content for any image.
    
    You will learn how to make a model that is similar to the one developed by Facebook, specifically such that a blind person knows the contents of an image in front of them with the help of a CNN-RNN based model. The model will convert the contents of an image and will provide the output in the form of audio.
   
   
 # <p style="background-color:white;font-family:seoge print;color:slateblue ;font-size:130%;text-align:center;border-radius:20px 60px;">Let's get started . . .</p>

<a id="1"></a>
# <p style="background-color:slateblue;font-family:newtimeroman;font-size:150%;color:white;text-align:center;border-radius:20px 60px;">CONTENTS</p>
----------------------------------------------------------------------------------------------------------


**1. Initial Setup**

        Import all required libraries
        
        Install additional libraries (if required)
        
        Mount drive and import dataset

**2. Data Understanding and Visualization**

        Unzip data files and read it's content
        
        Save the images and captions in two different folders (and variables)
        
        Visualize Images and Captions in the dataset
        
        Create dataframe to store images along with thir respective captions and path
        
        Visualize the top 30 most occurring words in the captions (through bar plots and wordcloud map)
        

**3. Data Cleaning**

        Remove punctuations
        
        Convert captions to lowercase
        
        Remove numeric values and retain words in captions
        
        
**4. Data Preprocessing**
 
   > **Captions Preprocessing**
 
         Create tokenizer (remove spaces and punctuation and retain unique words)
         
         Replace words beyond count of 5000 with tag : UNK
         
         Create word-to-index and index-to-word mappings
        
         Pad sequences to be of same length (as that of the longest)
         
         Finally view index with words and tokenizers
         
         
   > **Images Preprocessing**
   
          Resize images to shape : (299, 299)
           
          Normalize the image within the range of -1 to 1 (desired format for InceptionV3)
           
         
**5. Dataset Creation**

        1. Apply train_test_split on both image path & captions to create the train & test list. 

        2. Create a function which maps the image path to their feature.

        3. Create a builder function to create train & test dataset & transform the dataset
        
        4. Load the pretrained Imagenet weights of Inception net V3

        5. Shuffle and batch while building the dataset

        6. Ensure shape of each image in the dataset : (batch_size, 8*8, 2048)

        7. Ensureshape of each caption in the dataset : (batch_size, max_len)
        

**6. Model Building**

        Set model parameters (units, embed_dim, vocab_size, train & test num steps, feature_shape, max_length)
        
        Build Encoder
        
        Build Attention Model
         
        Build Decoder
        
        
**7. Model Training and Optimization**

        1.Set the optimizer & loss object

        2.Create your checkpoint path

        3.Create your training & testing step functions

        4.Create your loss function for the test dataset
        
**8. Model Evaluation**

        Set up evaluation function using Greedy Search
        
        Test on sample data images
        
        Evaluate on the basis of BLEU Score
        
        


# <p style="background-color:white;font-family:arial;font-size:150%;color:darkslateblue;text-align:center;border-radius:20px 60px;">Objective </p>

 > **Create a deep learning model which can explain the contents of an image in the form of speech through caption generation with an attention mechanism on Flickr8K dataset.**
 
 >**This kind of model is a use-case for blind people so that they can understand any image with the help of speech.** The caption generated through a CNN-RNN model will be converted to speech using a text to speech library. **The model will convert the contents of an image and will provide the output in the form of audio.**

## Techniques Used 

    1. Traditional CNN-RNN model

    2. Attention model

    3. Text and Image preprocessing

    4. Data Visualization using wordcloud

    5. Model training and model evaluation (INCEPTIONV3)

    6. Greedy search vs Beam search and BLUE Score
    


# Let's Dive in . . .
--------------------------------------------


<a id="1"></a>
# <p style="background-color:slateblue;font-family:newtimeroman;font-size:150%;color:white;text-align:center;border-radius:20px 60px;">Initial Setup</p>

In [ ]:
!nvidia-smi

### 1.1 Import all the required libraries

In [ ]:
#Import all the required libraries


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

import tensorflow as tf
import keras
from keras.preprocessing.image import load_img
import string
import time
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer

from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras import activations
from tensorflow.keras import Input
from PIL import Image

#used for creating Progress Meters or Progress Bars
from tqdm import tqdm


### 1.2 Install additional libraries required

We need to install the below libraries before proceeding :

**gTTS** - (Google Text-to-Speech)is a Python library and CLI tool to interface with Google Translate text-to-speech API. We will import the gTTS library from the gtts module which can be used for speech translation.

**playsound** - The playsound module is a cross platform module that can play audio files. This doesn't have any dependencies, simply install with pip in your virtualenv and run!

**display** - Public API for display tools in IPython

In [ ]:
# Install additional libraries required

!pip install wordcloud
!pip install gtts
!pip install playsound

In [ ]:
import glob
from gtts import gTTS
from playsound import playsound
from IPython import display

import collections
import wordcloud
from wordcloud import WordCloud, STOPWORDS


### 1.3 Mount drive and import dataset

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


<a id="1"></a>
# <p style="background-color:slateblue;font-family:newtimeroman;font-size:150%;color:white;text-align:center;border-radius:20px 60px;">Let's Understand our Data</p>

## 2. Data Understanding and Visualization

1.Import the dataset and read image & captions into two seperate variables

2.Visualise both the images & text present in the dataset

3.Create a dataframe which summarizes the image, path & captions as a dataframe

4.Create a list which contains all the captions & path

5.Visualise the top 30 occuring words in the captions



### 2.1 Import the dataset and read image & captions into two separate variables

In [ ]:
#Import the dataset and read the image into a seperate variable

images='/kaggle/input/flickr8k/Images'

all_imgs = glob.glob(images + '/*.jpg',recursive=True)
print("The total images present in the dataset: {}".format(len(all_imgs)))

### 2.2 Visualise both the images & text present in the dataset

In [ ]:
#Visualise both the images & text present in the dataset
import imageio

#Visualising first 5 images :

Display_Images = all_imgs[0:5]
figure, axes = plt.subplots(1,5)
figure.set_figwidth(20)

for ax, image in zip(axes, Display_Images):
  ax.imshow(imageio.imread(image), cmap=None)


In [ ]:
# view a random image

import random
Image.open(all_imgs[random.randrange(40, 60, 3)])

In [ ]:
#Import the dataset and read the text file into a separate variable

#txt_file = 'Flickr8K/captions.txt'
text_file = '/kaggle/input/flickr8k/captions.txt'

def load_doc(filename):
    
    #your code here
    open_file = open(text_file, 'r', encoding='latin-1' ) #returns a file object
    text = open_file.read() #reads contents of the file
    open_file.close()
    #print(text)
    
    return text

doc = load_doc(text_file)
print(doc[:300])

##  Create a dataframe which summarizes the image, path & captions as a dataframe

    Create a dataframe which summarizes the image, path & captions as a dataframe

    Each image id has 5 captions associated with it therefore the total dataset should have 40455 samples.

### 2.3 Create a list which contains all the captions & path

#### We will create three lists here and finally combine them to form a dataframe

    1. all_img_id = [] to store all the image id 
    2. all_img_vector = [] to store all the image path 
    3. annotations = []  to store all the captions 

In [ ]:
img_path = '/kaggle/input/flickr8k/Images/'

all_img_id = [] #store all the image id here
all_img_vector = [] #store all the image path here
annotations = [] #store all the captions here

with open('/kaggle/input/flickr8k/captions.txt' , 'r') as fo:
  next(fo) #to skip the heading
  for line in fo :
    split_arr = line.split(',')
    all_img_id.append(split_arr[0])
    annotations.append(split_arr[1].rstrip('\n.')) #removing out the \n.
    all_img_vector.append(img_path+split_arr[0])

df = pd.DataFrame(list(zip(all_img_id, all_img_vector,annotations)),columns =['ID','Path', 'Captions']) 
    
df

In [ ]:
len (annotations)

In [ ]:
type (annotations)

#### So now we have successfully created a list containing all captions as well as a dataframe which contains all images along withh thir respective paths and captions

   *Each image has 5 captions associated with it*
    
    Hence, total images : 8091
    
    Therefore, total captions : 8091*5 = 40455
    
**Let's verify this . .**    

In [ ]:
#check total captions and images present in dataset

print("Total captions present in the dataset: "+ str(len(annotations)))
print("Total images present in the dataset: " + str(len(all_imgs)))

In [ ]:
#Create the vocabulary & the counter for the captions
#lower() used to ensure same count irrespective of an alphabet's case

vocabulary = [word.lower() for line in annotations for word in line.split()]

val_count = Counter(vocabulary) 
val_count

### 2.5 Visualise the top 30 occuring words in the captions

In [ ]:
#Visualise the top 30 occuring words in the captions

#write your code here
for word, count in val_count.most_common(30):
  print(word, ": ", count)

lst = val_count.most_common(30)
most_common_words_df = pd.DataFrame(lst, columns = ['Word', 'Count'])
most_common_words_df.plot.bar(x='Word', y='Count', width=0.6, color='orange', figsize=(15, 10))
plt.title("Top 30 maximum frequency words", fontsize = 18, color= 'navy')
plt.xlabel("Words", fontsize = 14, color= 'navy')
plt.ylabel("Count", fontsize = 14, color= 'navy')


### Clearly STOPWORDS form a clear majority

**Stop words are a set of commonly used words in a language.**

    Examples of stop words in English are “a”, “the”, “is”, “are” and etc. 
    
*Stop words are commonly used in **Text Mining and Natural Language Processing (NLP)** to eliminate words that are so commonly used that they carry very little useful information.*

In [ ]:
## check out the top 30 stopwords with higher frequency

wordcloud = WordCloud(width = 1000, height = 500).generate_from_frequencies(val_count)
plt.figure(figsize = (12, 12))
plt.imshow(wordcloud)

### Visualize the images and captions together

In [ ]:
def caption_with_img_plot(image_id, frame) :
  #get the captions
  capt = ("\n" *2).join(frame[frame['ID'] == image_id].Captions.to_list())
  fig, ax = plt.subplots()
  ax.set_axis_off()
  idx = df.ID.to_list().index(image_id)
  im =  Image.open(df.Path.iloc[idx])
  w, h = im.size[0], im.size[-1]
  ax.imshow(im)
  ax.text(w+50, h, capt, fontsize = 18, color = 'navy')
caption_with_img_plot(df.ID.iloc[8049], df)

In [ ]:
def execute_img_capt(start, end, frame) :
  for r in range(start, end) :
    caption_with_img_plot(frame.ID.drop_duplicates().iloc[r], frame)

execute_img_capt(0, 5, df)  

<a id="3"></a>
# <p style="background-color:slateblue;font-family:newtimeroman;font-size:150%;color:white;text-align:center;border-radius:20px 60px;">Clean Data</p>

1. Remove punctuations
2. Convert captions to lowercase
3. Retain words and eliminate numeric values

In [ ]:
#data cleaning
rem_punct = str.maketrans('', '', string.punctuation)
for r in range(len(annotations)) :
  line = annotations[r]
  line = line.split()

  # converting to lowercase
  line = [word.lower() for word in line]

  # remove punctuation from each caption and hanging letters
  line = [word.translate(rem_punct) for word in line]
  line = [word for word in line if len(word) > 1]

  # remove numeric values
  line = [word for word in line if word.isalpha()]

  annotations[r] = ' '.join(line)


In [ ]:
#add the <start> & <end> token to all those captions as well
annotations = ['<start>' + ' ' + line + ' ' + '<end>' for line in annotations]

#Create a list which contains all the path to the images
all_img_path = all_img_vector

In [ ]:
##list contatining captions for an image
annotations[0:5]

<a id="4"></a>
# <p style="background-color:slateblue;font-family:newtimeroman;font-size:150%;color:white;text-align:center;border-radius:20px 60px;">Data Preprocessing</p>


### 4.1 Captions Preprocessing


Pre-Processing the captions

1. Create the tokenized vectors by tokenizing the captions fore ex :split them using spaces & other filters. 
This gives us a vocabulary of all of the unique words in the data. Keep the total vocaublary to top 5,000 words for saving memory.

2. Replace all other words with the unknown token "UNK" .

3. Create word-to-index and index-to-word mappings.

4. Pad all sequences to be the same length as the longest one.

In [ ]:
# create the tokenizer

#your code here

top_word_cnt = 5000
tokenizer = Tokenizer(num_words = top_word_cnt+1, filters= '!"#$%^&*()_+.,:;-?/~`{}[]|\=@ ',
                      lower = True, char_level = False, 
                      oov_token = 'UNK')

In [ ]:
# Create word-to-index and index-to-word mappings.

#your code here

tokenizer.fit_on_texts(annotations)

#transform each text into a sequence of integers
train_seqs = tokenizer.texts_to_sequences(annotations)

### Why Padding ?

 > Each individual image has **five captions which are all of different lengths**. Hence they **can't be fed directly to the Decoder.**
    
 > We will use **PADDING, to bring all captions to a fixed length** before proceeding further.

 > Padding is a process that we apply at the **start or the end of a sequence to make all the samples to have a common standard length.** 
    
 > For text generation, the **decoder needs to have an input at the start and it can't be a padded input.**
    
 > Therefore, for our process, we will be doing the **padding at the end** (of the caption sequence).

 > All **padded values will be masked and set to 'False' while the rest will be set to 'True'**
    
 > **All True values are assigned the value : 1, while padded, i.e False values are set to 0**
    
    
*Padding can result in a risk of adding penalty to the model.* 

*We apply masking to rectify the same and this will truncate down all the added penalties back to zero :*

    
### Loss  = Loss * Mask

As we will use pad_sequence to process the sequence, we will use 0 as the padding value.

 > Keras uses a different index for UNKNOWN, in order to distinguish between PAD and UNKNOWN

  > 0 is the reserved index which has no word assigned.

In [ ]:
# we add PAD token for zero

tokenizer.word_index['PAD'] = 0
tokenizer.index_word[0] = 'PAD'

In [ ]:
print(tokenizer.oov_token)
print(tokenizer.index_word[0])

#### View index words

In [ ]:
tokenizer.index_word

In [ ]:
# Create a word count of your tokenizer to visualize the Top 30 occuring words after text processing

#your code here
tokenizer_top_words = [word for line in annotations for word in line.split() ]

#tokenizer_top_words_count
tokenizer_top_words_count = collections.Counter(tokenizer_top_words)
tokenizer_top_words_count

In [ ]:
for word, count in tokenizer_top_words_count.most_common(30) :
  print(word, ": ", count)

tokens = tokenizer_top_words_count.most_common(30)
most_com_words_df = pd.DataFrame(tokens, columns = ['Word', 'Count'])

#plot 30 most common words
most_common_words_df.plot.bar(x = 'Word', y= 'Count', width=0.8, color = 'indigo', figsize = (17, 10))
plt.title('Top 30 common words', fontsize =20, color= 'navy')
plt.xlabel('Words', fontsize =14, color= 'navy')
plt.ylabel('Counts', fontsize =14, color= 'navy')
plt.grid(b=None)

### Let's view the frequency of words now, post Captions preprocessing

In [ ]:
wordcloud_token = WordCloud(width = 1000, height = 500).generate_from_frequencies(tokenizer_top_words_count)
plt.figure(figsize = (12, 8))
plt.imshow(wordcloud_token)
plt.grid(b = None)

In [ ]:
# Pad each vector to the max_length of the captions ^ store it to a vairable

#store the length of all lists
train_seqs_len = [len(seq) for seq in train_seqs]

#store elements from list with maximum value
longest_word_length = max(train_seqs_len)

#calculate longest word_length and pads all sequences to equal length as that of the longest.
cap_vector= tf.keras.preprocessing.sequence.pad_sequences(train_seqs, padding= 'post', maxlen = longest_word_length,
                                                          dtype='int32', value=0)

print("The shape of Caption vector is :" + str(cap_vector.shape))

**NOTE :** Using padding = 'post', we are padding zero at last. Starting word always acts as a crucial prompter and another word is expected after that. Zero at the beginning would indicate no value.

Hence in order to maintain existing sequence and it's meaning, we **apply PADDING at the end.**

## 4.2 Pre-processing the images

1.Resize them into the shape of (299, 299)

2.Normalize the image within the range of -1 to 1, such that it is in correct format for InceptionV3. 

### FAQs on how to resize the images::
* Since you have a list which contains all the image path, you need to first convert them to a dataset using <i>tf.data.Dataset.from_tensor_slices</i>. Once you have created a dataset consisting of image paths, you need to apply a function to the dataset which will apply the necessary preprocessing to each image. 
* This function should resize them and also should do the necessary preprocessing that it is in correct format for InceptionV3.


In [ ]:
# creating list to store preprocessed images and setting up the Image Shape

preprocessed_image = []
IMAGE_SHAPE = (299, 299)

In [ ]:
#checking image format 

tf.keras.backend.image_data_format()


**Image data format = 'channels_last' indicates Image is in three dimensional array, where third dim represents color channels (RGB)**

1st dim = rows

2nd dim = columns

3rd dim = channels 

In [ ]:
#write your code here for creating the function. This function should return images & their path

#write your pre-processing steps here (checking only for the first five images here)
for img in all_imgs[0:5] :
    img = tf.io.read_file(img, name=None)

    # we need to decode jpeg encoded images (here by default channels = 0)
    img = tf.image.decode_jpeg(img, channels=0)
    img = tf.image.resize(img, (299, 299))
    img = tf.keras.applications.inception_v3.preprocess_input(img)

    #append preprocessed images to the list
    preprocessed_image.append(img)


In [ ]:
# checking first five images post preprocessing

Display_Images = preprocessed_image[0:5]
figure, axes = plt.subplots(1,5)
figure.set_figwidth(25)

for ax, image in zip(axes, Display_Images) :
  print('Shape after resize : ', image.shape)
  ax.imshow(image)
  ax.grid('off')



<a id="5"></a>
# <p style="background-color:slateblue;font-family:newtimeroman;font-size:150%;color:white;text-align:center;border-radius:20px 60px;">Dataset Creation</p>

1.Create a function which maps the image path to their feature.

2.Create a builder function to create train & test dataset & apply the function created earlier to transform the dataset

3.Apply train_test_split on both image path & captions to create the train & test list. Create the train-test spliit using 80-20 ratio & random state = 42

2.Make sure you have done Shuffle and batch while building the dataset

3.The shape of each image in the dataset after building should be (batch_size, 8*8, 2048)

4.The shape of each caption in the dataset after building should be(batch_size, max_len)

**We will have a function here for preprocessing and returing images, so that we can use it for vectorization and preprocesses images parallelly**

In [ ]:
## write your code here for applying the function to the image path dataset,
## such that the transformed dataset should contain images & their path


def load_images(image_path) :
  img = tf.io.read_file(image_path, name = None)
  img = tf.image.decode_jpeg(img, channels=0)
  img = tf.image.resize(img, IMAGE_SHAPE)
  img = tf.keras.applications.inception_v3.preprocess_input(img)
  return img, image_path

In [ ]:
all_img_vector

In [ ]:
# Map each image full path to the function, in order to preprocess the image

## sort the unique paths and store in a list
training_list = sorted(set(all_img_vector))

#create a new dataset from above training list
New_Img = tf.data.Dataset.from_tensor_slices(training_list)

#map load_images function across the elements of the new dataset above
New_Img = New_Img.map(load_images, num_parallel_calls = tf.data.experimental.AUTOTUNE)
#Note  : Here, num_parallel_calls = tf.data.experimental.AUTOTUNE sets the number of parallel calls dynamically.
            ## based on the current GPU/CPU

#setting a batch size of 64
New_Img = New_Img.batch(64, drop_remainder=False)
#Note : As we don't want to drop the last batch if it contains less than 64 elements, we set drop_remainder to false

**Note :** As we didn't want to drop the last batch if it contains less than 64 elements, we **set drop_remainder to false**

In [ ]:
New_Img

### Train-test split

    Split ratio = 80 : 20

    Rabdom state = 42

In [ ]:
#Ratio = 80:20 and we will set random state = 42

path_train, path_test, caption_train, caption_test = train_test_split(all_img_vector, cap_vector, test_size = 0.2, random_state = 42)

In [ ]:
print("Training data for images: " + str(len(path_train)))
print("Testing data for images: " + str(len(path_test)))
print("Training data for Captions: " + str(len(caption_train)))
print("Testing data for Captions: " + str(len(caption_test)))

### Load the pretrained Imagenet weights of Inception net V3


    1.To save the memory(RAM) from getting exhausted, extract the features of the images using the last layer of pre-trained model. Including this as part of training will lead to higher computational time.

    2.The shape of the output of this layer is 8x8x2048. 

    3.Use a function to extract the features of each image in the train & test dataset such that the shape of each image should be (batch_size, 8*8, 2048)



In [ ]:
image_model = tf.keras.applications.InceptionV3(include_top=False,weights='imagenet')

new_input = image_model.input #write code here to get the input of the image_model
hidden_layer = image_model.layers[-1].output  #write code here to get the output of the image_model

#build the final model using both input & output layer
image_features_extract_model = tf.compat.v1.keras.Model(new_input, hidden_layer)

In [ ]:
# write the code to apply the feature_extraction model to your earlier created dataset which contained images & their respective paths
# Once the features are created, you need to reshape them such that feature shape is in order of (batch_size, 8*8, 2048)

image_features_extract_model.summary()

### NOTE : As we are not classifying the images here, there is no need to have the softmax layer at the end.

### FAQs on how to load the features:

* You can load the features using a dictionary created OR
* You can store using numpy(np.load) to load the feature vector.

In [ ]:
# extract features from each image in the dataset

img_features = {}
for image, image_path in tqdm(New_Img) :
  # we are using tqdm for progress bar

  # feed images from newly created Dataset above to Inception V3 built above
  batch_features = image_features_extract_model(image)
  #squeeze out the features in a batch
  batch_features_flattened = tf.reshape(batch_features, (batch_features.shape[0], -1, batch_features.shape[3]))

  for batch_feat, path in zip(batch_features_flattened, image_path) :
    feature_path = path.numpy().decode('utf-8')
    img_features[feature_path] = batch_feat.numpy()

In [ ]:
batch_features

In [ ]:
batch_features_flattened

In [ ]:
len(img_features)

In [ ]:
batch_feat.shape

In [ ]:
#view top five items of img_features dict
import more_itertools
top_5 = more_itertools.take(5, img_features.items())

top_5

### FAQs on how to store the features:
* You can store the features using a dictionary with the path as the key and values as the feature extracted by the inception net v3 model OR
* You can store using numpy(np.save) to store the resulting vector.

#### We will now feed both, the images as well as  the captions to the model

In [ ]:
#to provide, both images along with the captions as input
def map(image_name, caption):
    
    # your code goes here to create the dataset & transform it
    
    img_tensor = img_features[image_name.decode('utf-8')]
    return img_tensor, caption

#### We will set the bufffer size to 1000 and batch size to 64

    In Tensorflow, data is designed to work with huge sequences. Thus instead of shuffling entire sequence, it maintains separate buffer within which it shuffles elements. This buffer size of 1000 refers to the same.
    
    
    We will set reshuffle_each_iteration to True in order to ensure different order per epoch
    
    We will also use the prefetch function to prepare later elements as the current ones are being processed


In [ ]:
# create a builder function to create dataset which takes in the image path & captions as input
# This function should transform the created dataset(img_path,cap) to (features,cap) using the map_func created earlier

BUFFER_SIZE = 1000
BATCH_SIZE = 64
def gen_dataset(img, capt):
    
    # your code goes here to create the dataset & transform it
    
    data = tf.data.Dataset.from_tensor_slices((img, capt))
    # dataset created using tf.data.Dataset.from_tensor_slices
    data = data.map(lambda ele1, ele2 : tf.numpy_function(map, [ele1, ele2], [tf.float32, tf.int32]),
                    num_parallel_calls = tf.data.experimental.AUTOTUNE)
    
     
    data = (data.shuffle(BUFFER_SIZE, reshuffle_each_iteration= True).batch(BATCH_SIZE, drop_remainder = False)
    .prefetch(tf.data.experimental.AUTOTUNE))
    # .prefetch() is used to prepare all upcoming elements, while current elements are being processed
    # We set reshuffle_each_iteration set to True in order to ensure different order per epoch
    # Also,  drop_remainder is set to False as we don't want to miss out any element if the last batch contains less than 64 elements

    return data

In [ ]:
train_dataset = gen_dataset(path_train,caption_train)
test_dataset = gen_dataset(path_test,caption_test)

In [ ]:
sample_img_batch, sample_cap_batch = next(iter(train_dataset))
print(sample_img_batch.shape)  #(batch_size, 8*8, 2048)
print(sample_cap_batch.shape) #(batch_size,max_len)

<a id="1"></a>
# <p style="background-color:slateblue;font-family:newtimeroman;font-size:150%;color:white;text-align:center;border-radius:20px 60px;">Model Building</p>

    1.Set the parameters

    2.Build the Encoder

    3. Build the Attention Model

    4. Build the Decoder

In [ ]:
# Setting  parameters

embedding_dim = 256 
units = 512

#top 5,000 words +1
vocab_size = 5001
train_num_steps = len(path_train) // BATCH_SIZE #len(total train images) // BATCH_SIZE
test_num_steps = len(path_test) // BATCH_SIZE  #len(total test images) // BATCH_SIZE

max_length = 31
feature_shape = batch_feat.shape[1]
attention_feature_shape = batch_feat.shape[0]

## Encoder (CNN)

    1. The encoder parts involve the convolution of the input image with the help of various convolution, max pooling, and fully connected layers. 

    2. Since we are not dealing with the classification of the image, we have removed them from the end. 

    3. The final output of the encoder part will be the generation of the feature vector.

In [ ]:
tf.compat.v1.reset_default_graph()
print(tf.compat.v1.get_default_graph())

In [ ]:
#Building Encoder using CNN Keras subclassing method

class Encoder(Model):
    def __init__(self,embed_dim):
        super(Encoder, self).__init__()
        self.dense = tf.keras.layers.Dense(embed_dim) #build your Dense layer with relu activation
        
    def call(self, features):
        features =  self.dense(features) # extract the features from the image shape: (batch, 8*8, embed_dim)
        features =  tf.keras.activations.relu(features, alpha=0.01, max_value=None, threshold=0)
        return features

In [ ]:
encoder=Encoder(embedding_dim)

In [ ]:
from keras.utils.vis_utils import plot_model
#plot_model(encoder, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

### Note : 

 > The **CNN-based encoder produces the feature vector which is the encoded representation of the input image.**

 > The resulting **feature vector is static** and does not change at each timestamp.
    
 > Therefore we need to pass this vector to the **attention model along with the hidden state of the decoder to create the context vector**. 

## Attention model

#### Important Details : 

 > **Attention is an interface connecting the encoder and decoder that provides the decoder with information from every encoder hidden state.**
 
 > With this framework, the model is able to **selectively focus on valuable parts of the input sequence** and hence, learn the association between them.
 
 > The attention model produces an **output(context vector) that is fed to the decoder** for predicting the word at that timestamp
 
 > This output, i.e **context vector is adaptive in nature and change for each timestamp.**
 
 > It aims to overcome the limitation of traditional CNN-RNN based models. Using this, **instead of passing the complete input image to the RNN at every timestamp, we can pass different relevant parts of the image to it.**
 
 > This makes the model **faster and increase it's prediction accuracy.**

In [ ]:
class Attention_model(Model):
    def __init__(self, units):
        super(Attention_model, self).__init__()
        self.W1 = tf.keras.layers.Dense(units) #build your Dense layer
        self.W2 = tf.keras.layers.Dense(units) #build your Dense layer
        self.V = tf.keras.layers.Dense(1) #build your final Dense layer with unit 1
        self.units=units

    def call(self, features, hidden):
        # features shape: (batch_size, 8*8, embedding_dim)
        # hidden shape: (batch_size, hidden_size)

        # Expand the hidden shape to shape: (batch_size, 1, hidden_size)
        hidden_with_time_axis = hidden[:, tf.newaxis]

        # build your score funciton to shape: (batch_size, 8*8, units)
        score = tf.keras.activations.tanh(self.W1(features) + self.W2(hidden_with_time_axis))  

        # extract your attention weights with shape: (batch_size, 8*8, 1)
        attention_weights = tf.keras.activations.softmax(self.V(score), axis=1) 

        #shape: create the context vector with shape (batch_size, 8*8,embedding_dim)
        context_vector = attention_weights * features 

        # reduce the shape to (batch_size, embedding_dim)
        context_vector = tf.reduce_sum(context_vector, axis=1)  
        
        return context_vector, attention_weights

### Note 

    1. Attention model does a linear transformation of input. Hence we have aplied tanh on top of it, in order to introduce non-linearities and achieve a smoother distribution.
    
    2. The features produced from fully connected dense layer of encoder and the hidden state of decoder are fed to tanh function and attention score is calculated
    
    3. As we require an output in range (0,1), we will apply softmax function to attention score and get the final attention weights

## Decoder

    RNN - Model (GRU)

 > **Input to GRU : Context vector (from attention model) concatenated with  embedded vector (embedding layer)**. Output of this concatenation layer is fed to GRU as input
 
  > **Embedding layer present inside the decoder takes the input sequence** (preprocessed and transformed such that all samples have equal sequence length - through padding followed by masking). Embedding layer transforms this into an embedded vector.
  
  > **Concatenation layer contains : Embedded vector (output of embedding layer) along with the Context vector** (output of attention model)

In [ ]:
class Decoder(Model):
    def __init__(self, embed_dim, units, vocab_size):
        super(Decoder, self).__init__()
        self.units=units
        self.attention = Attention_model(self.units) #iniitalise your Attention model with units
        self.embed = tf.keras.layers.Embedding(vocab_size, embed_dim) #build your Embedding layer
        self.gru = tf.keras.layers.GRU(self.units,return_sequences=True,return_state=True,recurrent_initializer='glorot_uniform')
        self.d1 = tf.keras.layers.Dense(self.units) #build your Dense layer
        self.d2 = tf.keras.layers.Dense(vocab_size) #build your Dense layer
        

    def call(self,x,features, hidden):
        context_vector, attention_weights = self.attention(features, hidden) #create your context vector & attention weights from attention model
        embed = self.embed(x) # embed your input to shape: (batch_size, 1, embedding_dim)
        embed = tf.concat([tf.expand_dims(context_vector, 1), embed], axis = -1) # Concatenate your input with the context vector from attention layer. Shape: (batch_size, 1, embedding_dim + embedding_dim)
        output,state = self.gru(embed) # Extract the output & hidden state from GRU layer. Output shape : (batch_size, max_length, hidden_size)
        output = self.d1(output)
        output = tf.reshape(output, (-1, output.shape[2])) # shape : (batch_size * max_length, hidden_size)
        output = self.d2(output) # shape : (batch_size * max_length, vocab_size)
        
        return output, state, attention_weights
    
    def init_state(self, batch_size):
        return tf.zeros((batch_size, self.units))

In [ ]:
decoder=Decoder(embedding_dim, units, vocab_size)

In [ ]:
features=encoder(sample_img_batch)

hidden = decoder.init_state(batch_size=sample_cap_batch.shape[0])
dec_input = tf.expand_dims([tokenizer.word_index['<start>']] * sample_cap_batch.shape[0], 1)

predictions, hidden_out, attention_weights= decoder(dec_input, features, hidden)
print('Feature shape from Encoder: {}'.format(features.shape)) #(batch, 8*8, embed_dim)
print('Predcitions shape from Decoder: {}'.format(predictions.shape)) #(batch,vocab_size)
print('Attention weights shape from Decoder: {}'.format(attention_weights.shape)) #(batch, 8*8, embed_dim)

<a id="7"></a>
# <p style="background-color:slateblue;font-family:newtimeroman;font-size:150%;color:white;text-align:center;border-radius:20px 60px;">Model Training and Evaluation</p>
1.Set the optimizer & loss object

2.Create your checkpoint path

3.Create your training & testing step functions

4.Create your loss function for the test dataset

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001)  #define the optimizer
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True, reduction = tf.keras.losses.Reduction.NONE) #define your loss object

In [ ]:
def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    #loss is getting multiplied with mask to get an ideal shape
    
    return tf.reduce_mean(loss_)

### Why Masking ?

 *Padding can result in a risk of adding penalty to the model.* 

  *Once the padding is done, we need to apply 'masking'.*
    
 > Without masking, the model will **consider the padded input at that timestep, which will contribute to an increased loss.**
    
 > Through masking we need to inform the model to **ignore whenever a padded input is passed at a timestep**, hinting that this part of the input is padded. 
 


*We apply masking to rectify the same and this will truncate down all the added penalties back to zero :*

    
### Loss  = Loss * Mask


In [ ]:
checkpoint_path = "Flickr8K/checkpoint1"
ckpt = tf.train.Checkpoint(encoder=encoder,
                           decoder=decoder,
                           optimizer = optimizer)
ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

In [ ]:
start_epoch = 0
if ckpt_manager.latest_checkpoint:
    start_epoch = int(ckpt_manager.latest_checkpoint.split('-')[-1])

* While creating the training step for your model, you will apply Teacher forcing.


### Why Teacher Forcing ?

There are multiple issues with with training recurrent neural networks that use output from prior time steps as input :

    Slow convergence
    
    Model instability
    
    Poor skill

If the **previous output is incorrect (by any chance), it will result in inaccurate input for the next time stamp** which will further result in a different output than expected and the process will continue. 

**As a result, the model will get off track and will get punished for every subsequent word it generates. This makes learning slower and the model unstable.**

*To address this we  use Teacher Forcing*

 > Teacher forcing is a fast and effective way to train a recurrent neural network, where the **target/real word (i.e ground truth) is passed as the next input to the decoder instead of previous prediciton or output.**
 
 
 > Training with Teacher Forcing **converges faster.** At the early stages of training, the predictions of the model are very bad.
 
 > If we do not use Teacher Forcing, the hidden states of the model will be updated by a sequence of wrong predictions, errors will accumulate, and it is difficult for the model to learn from that.

In [ ]:
@tf.function
def train_step(img_tensor, target):
    loss = 0
    hidden = decoder.init_state(batch_size=target.shape[0])
    dec_input = tf.expand_dims([tokenizer.word_index['<start>']] * target.shape[0], 1)
    
    with tf.GradientTape() as tape:
        #write your code here to do the training steps
        encoder_op = encoder(img_tensor)

        #apply teacher forcing by passing target word as next input to the decoder
        for r in range(1, target.shape[1]) :
          #pass encoder_op to decoder
          predictions, hidden, _ = decoder(dec_input, encoder_op, hidden)
          loss = loss + loss_function(target[:, r], predictions) 
          dec_input = tf.expand_dims(target[:, r], 1)  

    avg_loss = (loss/ int(target.shape[1])) #avg loss per batch
    trainable_vars = encoder.trainable_variables + decoder.trainable_variables
    grad = tape.gradient (loss, trainable_vars) # calculating gradient wrt each trainable var

    #we will now compute the gradients and apply it to the optimizer while backpropagating
    optimizer.apply_gradients(zip(grad, trainable_vars))

    return loss, avg_loss

* While creating the test step for your model, you will pass your previous prediciton as the next input to the decoder.

In [ ]:
@tf.function
def test_step(img_tensor, target):
    loss = 0
    
    #write your code here to do the testing steps
    hidden = decoder.init_state(batch_size = target.shape[0])

    dec_input = tf.expand_dims([tokenizer.word_index['<start>']] * target.shape[0], 1)
    with tf.GradientTape() as tape:
      encoder_op = encoder(img_tensor)

      #apply teacher forcing again
      for r in range(1, target.shape[1]) :
        #pass encoder_op to decoder
        predictions, hidden, _ = decoder(dec_input, encoder_op, hidden)
        loss = loss + loss_function(target[:, r], predictions)

        dec_input = tf.expand_dims(target[: , r], 1)

    avg_loss = (loss/ int(target.shape[1])) #avg loss per batch
    trainable_vars = encoder.trainable_variables + decoder.trainable_variables
    grad = tape.gradient (loss, trainable_vars) # calculating gradient wrt each trainable var

    #we will now compute the gradients and apply it to the optimizer while backpropagating
    optimizer.apply_gradients(zip(grad, trainable_vars))                      


    return loss, avg_loss

In [ ]:
def test_loss_cal(test_dataset):
    total_loss = 0

    #write your code to get the average loss result on your test data
    for (batch, (img_tensor, target)) in enumerate(test_dataset) :
      batch_loss, t_loss = test_step(img_tensor, target)
      total_loss = total_loss + t_loss
      avg_test_loss = total_loss/ test_num_steps

    return avg_test_loss

In [ ]:
loss_plot = []
test_loss_plot = []
EPOCHS = 15

best_test_loss=100
for epoch in tqdm(range(0, EPOCHS)):
    start = time.time()
    total_loss = 0

    for (batch, (img_tensor, target)) in enumerate(train_dataset):
        batch_loss, t_loss = train_step(img_tensor, target)
        total_loss += t_loss
        avg_train_loss=total_loss / train_num_steps
        
    loss_plot.append(avg_train_loss)    
    test_loss = test_loss_cal(test_dataset)
    test_loss_plot.append(test_loss)
    
    print ('For epoch: {}, the train loss is {:.3f}, & test loss is {:.3f}'.format(epoch+1,avg_train_loss,test_loss))
    print ('Time taken for 1 epoch {} sec\n'.format(time.time() - start))
    
    if test_loss < best_test_loss:
        print('Test loss has been reduced from %.3f to %.3f' % (best_test_loss, test_loss))
        best_test_loss = test_loss
        ckpt_manager.save()

### Final Test loss after 15 epochs = 0.482

*Let us now plot the training vs test loss and check the trend*

In [ ]:
from matplotlib.pyplot import figure
figure(figsize=(12, 8))
plt.plot(loss_plot, color='orange', label = 'training_loss_plot')
plt.plot(test_loss_plot, color='green', label = 'test_loss_plot')
plt.xlabel('Epochs', fontsize = 15, color = 'red')
plt.ylabel('Loss', fontsize = 15, color = 'red')
plt.title('Loss Plot', fontsize = 20, color = 'red')
plt.legend()
plt.show()


 > **Test Loss : Indicated by the green line, shows a steady pattern along with training loss (indicated by orange), despite the use of teacher forcing**

 > Also, we will **stick to 15 approaches and not increase them further, as the intent here is not to create the state of art model. Rather understand how to integrate attention mechanism with E-D architecture for images.**

### NOTE : 

* Since there is a difference between the train & test steps (Presence of teacher forcing), you may observe that the train loss is decreasing while your test loss is not. 

* This doesn't mean that the model is overfitting, as we can't compare the train & test results here, as both approach is different.


<a id="1"></a>
# <p style="background-color:slateblue;font-family:newtimeroman;font-size:150%;color:white;text-align:center;border-radius:20px 60px;">Final Model Evaluation</p>
1.Define your evaluation function using greedy search

2.Define your evaluation function using beam search ( optional)

3.Test it on a sample data using BLEU score

### Greedy Search

 > This method is a simple approximation technique which calculates the **probability of the words according to their occurrence in the English vocabulary.**
 
 > It **takes the sample of the words, finds the probability of each of the words, and then outputs the word with the highest probability.**
 
 > Greedy Search will always consider **only one best alternative and this makes the computational speed of the model fast**, but the accuracy might not be up to the mark.

In [ ]:
def evaluate(image):
    attention_plot = np.zeros((max_length, attention_feature_shape))

    hidden = decoder.init_state(batch_size=1)

    temp_input = tf.expand_dims(load_images(image)[0], 0) #process the input image to desired format before extracting features
    img_tensor_val = image_features_extract_model(temp_input) # Extract features using our feature extraction model
    img_tensor_val = tf.reshape(img_tensor_val, (img_tensor_val.shape[0], -1, img_tensor_val.shape[3]))

    features = encoder (img_tensor_val) # extract the features by passing the input to encoder

    dec_input = tf.expand_dims([tokenizer.word_index['<start>']], 0)
    result = []

    for i in range(max_length):
        predictions, hidden, attention_weights = decoder(dec_input, features, hidden) # get the output from decoder

        attention_plot[i] = tf.reshape(attention_weights, (-1, )).numpy()

        predicted_id = tf.argmax(predictions[0]).numpy() #extract the predicted id(embedded value) which carries the max value
        #map the id to the word from tokenizer and append the value to the result list
        result.append (tokenizer.index_word[predicted_id])

        if tokenizer.index_word[predicted_id] == '<end>':
            return result, attention_plot,predictions

        dec_input = tf.expand_dims([predicted_id], 0)

    attention_plot = attention_plot[:len(result), :]
    return result, attention_plot,predictions


In [ ]:
def plot_attention_map (caption, weights, image) :

  fig = plt.figure(figsize = (10, 10))
  temp_img = np.array(Image.open(image))

  cap_len = len(caption)
  for cap in range(cap_len) :
    weights_img = np.reshape(weights[cap], (8,8))
    wweights_img = np.array(Image.fromarray(weights_img).resize((224,224), Image.LANCZOS))

    ax = fig.add_subplot(cap_len//2, cap_len//2, cap+1)
    ax.set_title(caption[cap], fontsize = 14, color = 'red')

    img = ax.imshow(temp_img)

    ax.imshow(weights_img, cmap='gist_heat', alpha=0.6, extent=img.get_extent())
    ax.axis('off')
  plt.subplots_adjust(hspace=0.2, wspace=0.2)
  plt.show()

In [ ]:
from nltk.translate.bleu_score import sentence_bleu

In [ ]:
def filt_text(text):
    filt=['<start>','<unk>','<end>'] 
    temp= text.split()
    [temp.remove(j) for k in filt for j in temp if k==j]
    text=' '.join(temp)
    return text

In [ ]:
image_test = path_test.copy()

In [ ]:
def pred_caption_audio(random, autoplay=False, weights=(0.5, 0.5, 0, 0)) :

    cap_test_data = caption_test.copy()
    rid = np.random.randint(0, random)
    test_image = image_test[rid]
    #test_image = './images/413231421_43833a11f5.jpg'
    #real_caption = '<start> black dog is digging in the snow <end>'

    real_caption = ' '.join([tokenizer.index_word[i] for i in cap_test_data[rid] if i not in [0]])
    result, attention_plot, pred_test = evaluate(test_image)


    real_caption=filt_text(real_caption)      


    pred_caption=' '.join(result).rsplit(' ', 1)[0]


    real_appn = []
    real_appn.append(real_caption.split())
    reference = real_appn
    candidate = pred_caption.split()

    score = sentence_bleu(reference, candidate, weights=weights)#set your weights
    print(f"BELU score: {score*100}")
    print ('Real Caption:', real_caption)
    print ('Prediction Caption:', pred_caption)
    plot_attention_map(result, attention_plot, test_image)

    # we will make use of Google Text to Speech API (online), which will convert the caption to audio
    speech = gTTS('Predicted Caption : ' + pred_caption, lang = 'en', slow = False)
    speech.save('voice.mp3')
    audio_file = 'voice.mp3'

    display.display(display.Audio(audio_file, rate = None, autoplay = autoplay))

    return test_image
    


<a id="8"></a>
# <p style="background-color:slateblue;font-family:newtimeroman;font-size:150%;color:white;text-align:center;border-radius:20px 60px;">Model Testing</p>

It is finally time to check how our model performs on Images. 

 > **Our evaluation metric : BLUE SCORE** (Bilingual Evaluation Understudy)
 
  ### Note : 
  
    1.  BLEU is a well-acknowledged metric to measure similarity of one hypothesis sentence to multiple reference sentences. Given a single hypothesis sentence and multiple reference sentences, it returns a value between 0 and 1. 
    
  **The metric close to 1 means that the two are very similar.**
    
    2. We use the BLEU measure to evaluate the result of the test set generated captions. The BLEU is simply taking the fraction of n-grams in the predicted sentence that appears in the ground-truth.


# <p style="background-color:white;font-family:seoge print;color:slateblue ;font-size:100%;text-align:left;border-radius:20px 60px;">Test Image 1</p>

In [ ]:
test_image = pred_caption_audio(len(image_test), True, weights = (0.5, 0.25, 0, 0))
Image.open(test_image)

### WOW!! 100% accuracy as indicated by BLEU Score

# <p style="background-color:white;font-family:seoge print;color:slateblue ;font-size:100%;text-align:left;border-radius:20px 60px;">Test Image 2</p>

In [ ]:
test_image = pred_caption_audio(len(image_test), True, weights = (0.5, 0.25, 0, 0))
Image.open(test_image)

# <p style="background-color:white;font-family:seoge print;color:slateblue ;font-size:100%;text-align:left;border-radius:20px 60px;">Test Image 3</p>

In [ ]:
test_image = pred_caption_audio(len(image_test), True, weights = (0.5, 0.25, 0, 0))
Image.open(test_image)

#### 100% accuracy yet again! So this combination of weights actually work better than others.

# <p style="background-color:white;font-family:seoge print;color:slateblue ;font-size:100%;text-align:left;border-radius:20px 60px;">Test Image 4</p>

In [ ]:
test_image = pred_caption_audio(len(image_test), True, weights = (0.35, 0.25, 0, 0))
Image.open(test_image)

# <p style="background-color:white;font-family:seoge print;color:slateblue ;font-size:100%;text-align:left;border-radius:20px 60px;">Test Image 5</p>

In [ ]:
test_image = pred_caption_audio(len(image_test), True, weights = (0.5, 0.5, 0, 0))
Image.open(test_image)

# <p style="background-color:white;font-family:seoge print;color:slateblue ;font-size:100%;text-align:left;border-radius:20px 60px;">Test Image 6</p>

In [ ]:
test_image = pred_caption_audio(len(image_test), True, weights = (0.25, 0.5, 0, 0))
Image.open(test_image)

# <p style="background-color:white;font-family:seoge print;color:slateblue ;font-size:100%;text-align:left;border-radius:20px 60px;">Test Image 7</p>

In [ ]:
test_image = pred_caption_audio(len(image_test), True, weights = (0.25, 0.25, 0, 0))
Image.open(test_image)

# <p style="background-color:white;font-family:seoge print;color:slateblue ;font-size:100%;text-align:left;border-radius:20px 60px;">Test Image 8</p>

In [ ]:
test_image = pred_caption_audio(len(image_test), True, weights = (0.25, 0.5, 0, 0))
Image.open(test_image)


<a id="9"></a>
# <p style="background-color:slateblue;font-family:newtimeroman;font-size:150%;color:white;text-align:center;border-radius:20px 60px;">Conclusion</p>


### 1. We experimented with multiple other weights (12-15 different combinations) but found the best results mainly for the below :

    1. (0.5, 0.5 , 0, 0)
    
    2. (0.5, 0.25 , 0, 0)
    
    3. (0.25, 0.25 , 0, 0)
    
    4. (0.25, 0.35 , 0, 0)
    
 > The probability of getting a **lower Bleu (less than 50%) score was least for the above weights.**
 
 > We achieved **more than 70% accuracy for majority of the test images using above weights.**
 
 > We also got an **accuracy of 100%  multiple time using weights : (0.5, 0.5 , 0, 0), (0.5, 0.25, 0, 0) and (0.25, 0.5, 0, 0)**
 
 
 ### 2. We used Inception V3 Model with pretrained weights (Imagenet), for CNN (to extract feature vectors) 
 
    1. Inception v3 is a widely-used image recognition model that has been shown to attain greater than 78.1% accuracy on the ImageNet dataset
     
     2. The model is the culmination of many ideas developed by multiple researchers over the years.
     
     3. The model itself is made up of symmetric and asymmetric building blocks, including convolutions, average pooling, max pooling, concatinations, dropouts, and fully connected layers. 
     
     4. Batch normalization is used extensively throughout the model and applied to activation inputs. Loss is computed using Softmax.
     
### 3. GRU has been used here for the Decoder (RNN)

    1. Attention Model has also been used in Decoder to ensure higher focus on relevant and particular parts of the image at a given timestamp, rather than focussing on entire image.
    
    2. This ensured a higher accuracy, reduced noise and faster computation.
    
    3. The output from the decoder - i.e predicted caption and  hidden state, is fed back to the model and predictions were used to calculate the loss.
    
    4. We used cross entropy - SparseCategoricalCrossentropy to calculate the loss.
    
    5. We also made use of Teacher forcing to decide the next input to the decoder. It ensured faster convergence and a reliably stable model.
    
 ### 4. Model Training and  Evaluation
 
     1. We trained the model in 15 EPOCHS and the final loss got reduced to 0.482.
     
     2. We chose not to use further epochs as the intent here was not to create the state of art model. Rather understand how to integrate attention mechanism with E-D architecture for images.
 
 
     3. Prediction of Captions is done through calculating the probabilities of a specific word in the vocabulary.
     
     4. We made use of Greedy Search to calculate the probability of words as per their occurrence in the given vocabulary list. It outputs the word with the highest probability.
     
     6. We made use of BLUE Score to evaluate our model performance and accuracy. Higher the BLUE score, better the model performance.
     
     7. We tested the model with different combinations of weights and achieve the max Bleu Score (100%) for several combinations.
     
     
 
 ----------------------------------------------------------

![](https://voximplant.com/assets/images/2020/07/03/speech-to-text.png)